In [ ]:
# Import libraries
import pandas as pd
import requests, io, json
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
from datetime import datetime
from typing import Dict, List

In [ ]:
# Step 1 - Call the API for available language models
import requests
request = requests.get(f"http://cmdb.vectorspacebio.science/api/get_language_models?vxv_wallet_addr=0076e0b8328203a1beb056caf17a05a78929bd74")
language_models = request.json()
print(language_models)

In [ ]:
# Step 2 - Choose a language model - e.g., Biosciences_0001
language_model = language_models[0]
print(language_model)

In [ ]:
# Step 3 - Define up to 10 custom features / column labels (up to 100,000 labels upon request)
column_labels = ["mena", "opa1", "cardiolipin", "plant"]

In [ ]:
# Step 4 - Optional: Submit a context-dependency - e.g., "mitochondria"
context_dependency = ""
# context_dependency = "mitochondria"

In [ ]:
# Step 5 - Call the API with the selected parameters and return two matrices
# This method may take several minutes to complete depending on the request.
import requests
import pandas as pd

# Specify the types of rows
row_types = "protein_names"

# Add the context dependency component of the query
context_dependency_param = ""
if context_dependency != "":
    context_dependency_param = f"&context_dependency={context_dependency}"

# Retrieve the correlation matrix dataset and hidden relationship matrix
cmd_request = requests.get(f"http://cmdb.vectorspacebio.science/api/get_correlation_matrix?vxv_wallet_addr=0076e0b8328203a1beb056caf17a05a78929bd74&language_model={language_model['name']}&row_types={row_types}&column_labels={'%2C%20'.join(column_labels)}{context_dependency_param}")

# Retrieve request and convert to dataframes
cm_dataset = pd.DataFrame.from_dict(cmd_request.json())

# Preview the correlation matrix dataset
cm_dataset.head()

In [ ]:
# Example of heatmap with rows (proteins) and column labels

from IPython.display import Image
Image(filename='heatmap.jpg') 

In [ ]:
# Step 6 - Create a graph from the correlation matrix dataset
# To expand the network of relationships, submit each leaf node to the PPIN (Protein-Protein Interaction Network) API
# The number of leaf nodes can be increased by adjusting the value from 5
import requests

max_depth = 1
top_n = 5

network_results = []

for column_label in list(cm_dataset.keys()):
    ppin_request = requests.get(f"http://ppin.vectorspacebio.science/api/protein_network?vxv_wallet_addr=d1030ededff0f348992fd4d41ae169dae99bf9c0&query={column_label}&branches={top_n}&max_depth={max_depth}")
    result = json.loads(ppin_request.content.decode())
    network_results.append(result)

In [ ]:
# Step 7 - Prepare the data for high charts or similar software
nodes = []

# Add the root node of the graph
root_protein = "klf4"
for col in column_labels:
    nodes.append([[root_protein, col]])

# Add the leaf nodes
for result in network_results:
    for item in result["graph_network"]:
        nodes.append([[item["source"], item["target"]], item["hidden_relationship"]])

# Output nodes to go into graph software
print(nodes)